In [1]:
%matplotlib inline
import math
import numpy as np
import random
import matplotlib.pyplot as plt
import re
import os.path
from scipy.spatial import distance

In [3]:
'''
    preprocess the dataset
    if preprocessed data exist(new_data_1),read in new_data_1;
    otherwise read and prepocess the adult.data.txt
'''

# dimension of each example in the dataset
d = 15
# k is the top pricipal calculated by the eigenvectors, right now just fix it
k = 5
# preprocess the data
data_set = []
data_set_done = []
if os.path.exists('new_data_1'):
    print "reading in existing file"
    with open('new_data_1') as data:
        for line in data:
            data_set_done.append(map(float, line.split()))
else:
    with open('adult.data.txt') as data:
        for line in data:
            # '\s' matches whitespace
            tmp = re.sub(r'\s', '', line).split(',')
            data_set.append(tmp)
    #print data_set
    feature_vector_size = len(data_set[0])
    print "feature vector is  " ,feature_vector_size
    print "data set size is ", len(data_set)
    tmp = data_set[0]
    
    # find out which feature is not a digit
    non_digit_index = []
    for i in range (0,feature_vector_size):
        if tmp[i].isdigit():
            continue
        else:
            non_digit_index.append(i)
    print non_digit_index

    modified_data_set= []
    for item in data_set:
        modified_data_set.append(item)
        
    print "data before processed dimension is ", len(modified_data_set[0])
    item_list = []
    count = 0
    dimension_to_increase = 0
    # for all the non_digit index, count how many differnt items
    for i in non_digit_index:
        # clear the item list to find different string for same feature
        items = []
        # loop through the whole data set
        for vector in data_set:
            # get the corresponsding item
            tmp = vector[i]
            # it already in the list
            if tmp in items:
                continue
            else:
                # otherwise append to the list
                items.append(tmp)
        dimension_to_increase = dimension_to_increase + len(items)-1
        # loop through the whole data set to replace the feature
        for j in range(len(modified_data_set)):
            v = modified_data_set[j]
            # get the item in the vector
            tmp = v[i+count]
            index  = items.index(tmp)
            front = v[0:i+count]
            end = v[i+1+count:len(v)]
            zeros = [0]*len(items)
            zeros[index] = 1
            front.extend(zeros)
            front.extend(end)
            modified_data_set[j] = front
            #v[i] = float(index)
        count = count + len(items)-1
    for v in modified_data_set:
        v = map(float, v)
        data_set_done.append(v)
    print "dimension to increase ", dimension_to_increase
    print "data processed dimension is ", len(data_set_done[0])
    #print data_set_done
    print "size of data set done is ", len(data_set_done)

    # write to new data 1 
    f = open('new_data_1', 'w')
    for vector in data_set_done:
        for num in vector:
            f.write(str(num)+" ")
        f.write("\n")
    f.close()
    print "reading data set done"

reading in existing file


In [4]:
'''
    helper function to normalize a vector
    v = np.array([[x1,x2]])
    row-major vectors
'''
def normalizeVector(v):
    distance = 0.0
    for i in v:
        for j in i:
            distance = distance + j**2
    distance = math.sqrt(distance)
    v = v/distance
    return v

# helper function to calculate the distance between two vectors
# v1 = np.array([[x1,x2]])
# v2 = np.array([[x1,x2]])
def distance(v1,v2):
    distance = 0.0
    for i,j in zip(v1,v2):
        for x, y in zip(i, j):
            distance = distance + (x- y)**2
    distance = math.sqrt(distance)
    return distance

In [56]:
# v is np.array([[x1,x2,x3....]])
# v is column-major vector
# from 5/4 notes

def testFunction(v):
    total = 0.0
    # loop through every data points
    for item in data_set_done:
        tmp = np.array([item])
        dotProduct = np.dot(tmp,v) # dotproduct should be a constant
        #print "dotProduct is ", dotProduct
        dotProduct = dotProduct * v # dotproduct is a column major vector
        #print "dotProduct is ", dotProduct
        total = total + distance(tmp,dotProduct)**2
    return total

In [21]:
'''
    Calculate convarience matrix M
    M is size 110 * 110
'''

N = len(data_set_done)
M = np.array(data_set_done[0]) * np.transpose(np.array(data_set_done[0]))

for i in range(1, len(data_set_done)):
    M = M + (np.array([data_set_done[i]]) * np.transpose(np.array([data_set_done[i]])))
print "N is ", N 
print "M is "
print M
print "M shape should be 110 * 110: ",M.shape

# divide M by N accroding to the formula on the notes
M_new = np.divide(M, N)
print M_new
print "M_new shape should be 110 * 110 ", M_new.shape

N is  32561
M is 
[[  6.73168185e+03   5.69322222e+02   1.26964444e+03 ...,   3.55555556e-01
    1.01038333e+04   3.85514444e+03]
 [  5.68510000e+02   1.29800000e+03   0.00000000e+00 ...,   0.00000000e+00
    9.45000000e+02   3.53000000e+02]
 [  1.26983222e+03   1.00000000e+00   2.54100000e+03 ...,   0.00000000e+00
    1.81800000e+03   7.24000000e+02]
 ..., 
 [  5.43333333e-01   1.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    2.00000000e+00   0.00000000e+00]
 [  1.01030211e+04   9.45000000e+02   1.81700000e+03 ...,   1.00000000e+00
    2.47200000e+04   0.00000000e+00]
 [  3.85533222e+03   3.54000000e+02   7.24000000e+02 ...,   0.00000000e+00
    1.00000000e+00   7.84100000e+03]]
M shape should be 110 * 110:  (110, 110)
[[  2.06740636e-01   1.74847892e-02   3.89927964e-02 ...,   1.09196755e-05
    3.10304761e-01   1.18397606e-01]
 [  1.74598446e-02   3.98636406e-02   0.00000000e+00 ...,   0.00000000e+00
    2.90224502e-02   1.08411904e-02]
 [  3.89985634e-02   3.07115875e-05  

In [28]:
'''
    baseline: SVD result
'''
U, s, V = np.linalg.svd(M_new, full_matrices=True)
print "U, ", U
print "s, ", s
print "V, ", V

U,  [[ -1.90463850e-01  -3.31716402e-02  -9.40799429e-02 ...,   9.07728753e-14
   -4.20549012e-14  -1.25038868e-14]
 [ -1.59447295e-02  -7.67494025e-05  -2.97051343e-02 ...,   5.26999009e-02
   -9.53648556e-02   8.11722158e-02]
 [ -3.33624096e-02  -4.04214384e-02   1.44485450e-03 ...,   5.26999009e-02
   -9.53648556e-02   8.11722158e-02]
 ..., 
 [ -2.38329083e-05   5.81954644e-05   7.73638063e-06 ...,  -2.92250721e-02
   -5.71792914e-03  -4.57065996e-02]
 [ -3.37129107e-01   3.13629887e-01   2.93413632e-01 ...,   1.30655494e-01
    2.64647694e-01   1.09576004e-01]
 [ -1.06136541e-01  -2.62488772e-01  -3.66689143e-01 ...,   1.30655494e-01
    2.64647694e-01   1.09576004e-01]]
s,  [  4.99047579e+00   9.47494033e-01   3.84173393e-01   3.28023626e-01
   2.60877129e-01   2.58519959e-01   2.20754550e-01   1.84105011e-01
   1.69024545e-01   1.46673458e-01   1.30185715e-01   1.14914977e-01
   1.12559572e-01   1.07092625e-01   1.04557541e-01   9.01387434e-02
   8.81591695e-02   8.39462052e-02  

In [62]:
# use the first column of U returned by np.linalg.svd
print np.transpose(np.array([U[:,0]]))
print "baseline result is ", testFunction(np.transpose(np.array([U[:,0]])))

[[ -1.90463850e-01]
 [ -1.59447295e-02]
 [ -3.33624096e-02]
 [ -3.20211786e-01]
 [ -1.16434912e-02]
 [ -2.57600092e-02]
 [ -2.17146962e-02]
 [ -1.44399891e-02]
 [ -1.87578561e-04]
 [ -9.09148083e-05]
 [ -5.65239071e-02]
 [ -7.38536267e-02]
 [ -1.48134939e-01]
 [ -1.50901445e-02]
 [ -2.29182422e-02]
 [ -6.40036338e-03]
 [ -9.93676791e-02]
 [ -1.41752787e-02]
 [ -1.86027094e-02]
 [ -8.13603579e-03]
 [ -5.43595738e-03]
 [ -7.68370049e-03]
 [ -3.78139187e-03]
 [ -1.19632857e-02]
 [ -1.87236150e-03]
 [ -5.66394678e-04]
 [ -5.46345074e-03]
 [ -2.80286577e-01]
 [ -1.41944355e-01]
 [ -2.15012114e-01]
 [ -5.71642915e-02]
 [ -4.73349094e-03]
 [ -1.22537037e-02]
 [ -2.97606900e-04]
 [ -1.19243398e-02]
 [ -4.96454565e-02]
 [ -5.64117269e-02]
 [ -1.90740250e-02]
 [ -5.53790197e-02]
 [ -4.27690063e-02]
 [ -5.11337143e-02]
 [ -5.97700185e-02]
 [ -2.29972166e-02]
 [ -1.36969802e-02]
 [ -2.78268111e-02]
 [ -1.25775973e-02]
 [ -2.17927601e-02]
 [ -8.49835002e-03]
 [ -1.25903890e-04]
 [ -1.73412339e-03]


In [58]:
# use the first row of V returned by np.linalg.svd
print np.array(np.transpose([V[0]]))
print "baseline result is ", testFunction(np.array(np.transpose([V[0]])))

[[ -1.90456877e-01]
 [ -1.59606638e-02]
 [ -3.33496577e-02]
 [ -3.20199167e-01]
 [ -1.16306543e-02]
 [ -2.57472355e-02]
 [ -2.17019239e-02]
 [ -1.44272306e-02]
 [ -1.74728219e-04]
 [ -7.80637661e-05]
 [ -5.65111982e-02]
 [ -7.38691091e-02]
 [ -1.48122408e-01]
 [ -1.50773167e-02]
 [ -2.29055468e-02]
 [ -6.38755273e-03]
 [ -9.93549224e-02]
 [ -1.41624543e-02]
 [ -1.85899243e-02]
 [ -8.12324214e-03]
 [ -5.42315838e-03]
 [ -7.67093084e-03]
 [ -3.76859523e-03]
 [ -1.19504672e-02]
 [ -1.85953990e-03]
 [ -5.53549632e-04]
 [ -5.45060707e-03]
 [ -2.80292963e-01]
 [ -1.41958956e-01]
 [ -2.15000773e-01]
 [ -5.71516002e-02]
 [ -4.72068516e-03]
 [ -1.22409635e-02]
 [ -2.84759056e-04]
 [ -1.19115886e-02]
 [ -4.96611505e-02]
 [ -5.63990597e-02]
 [ -1.90611456e-02]
 [ -5.53662361e-02]
 [ -4.27563041e-02]
 [ -5.11209426e-02]
 [ -5.97572688e-02]
 [ -2.29844098e-02]
 [ -1.36841276e-02]
 [ -2.78140845e-02]
 [ -1.25647554e-02]
 [ -2.17799877e-02]
 [ -8.48550575e-03]
 [ -1.13051895e-04]
 [ -1.72129515e-03]


In [65]:
# random shuffle the dataset
np.random.shuffle(data_set_done)

In [100]:
'''
    Implement Oja's rule to find the top k pricipal components of the dataset
'''
# initialize v0 as a vector drawn uniformly at raondom on the surface of a d-dimensional sphere
d = len(data_set_done[0])
v1 = []
prev_v = [] # record the previous v1 to test convergence
s = 0.0
t = len(data_set_done) # t is the data set lenght: how many data points there are
learning_rate = 1.0
print "learning rate is ",learning_rate

# random intialize a vector from a d-dim sphere
for i in range(0,d):
    tmp = np.random.random()
    s = s + tmp**2 # for normalization
    v1.append(tmp)
    prev_v.append(0.0)

s = math.sqrt(s)

# normalize v1 
for i in range(0,d):
    v1[i] = v1[i] * 1.0/ s

#a = np.dot(np.array([data_set_done[0]]),np.transpose(np.array([data_set_done[0]])))

dis = 0.0 # use distance to test convergence
loop = 0.0
v1 = np.array([v1]) # v1 type np.array([[x1,x2]]) 
v1 = np.transpose(v1) # let v1 becomese a column major vector
print "intialized v1 is ",v1
prev_v = v1
testResult = []
tPoint = []
index = 1

# while True:
#     loop = loop + 1
#     testResult = []
#     tPoint = []
#     #print "loop is ", loop
#     for i in range(0, len(data_set_done)):
#     #for i in range(0, len(data_set_done)):
#         learning_rate = 1.0/math.sqrt(index)# fix the leanring rate problem
#         index = index + 1
#         tmp = data_set_done[i]
#         tmp = np.array([tmp])
#         b = np.dot(np.transpose(tmp),tmp) # result in 110 * 110
#         b = np.dot(b,v1) # result will be a 110 * 1 vector
#         v1 = v1 + learning_rate * b 
#         v1 = normalizeVector(v1)
#         if i % 20 == 0:
#             tPoint.append(i)
#             test = testFunction(v1)
#             # print "test " ,test
#             testResult.append(test)
#     print testResult
#     plt.plot(tPoint, testResult, '-')
#     #plt.axis([0, len(data_set_done), 0, 5000])
#     plt.ylabel('test function result')
#     plt.show()
#     #print "global v1, ", v1
#     dis = distance(prev_v,v1)
#     print "distance ", dis
#     if dis <= 0.001:
#         break
#     prev_v = v1

learning rate is  1.0
intialized v1 is  [[  1.18393937e-01]
 [  4.35110900e-02]
 [  1.29056891e-01]
 [  1.60353070e-01]
 [  1.53917481e-02]
 [  4.01601069e-02]
 [  7.86243162e-02]
 [  8.91802072e-02]
 [  2.60132418e-03]
 [  1.02866304e-01]
 [  1.13983905e-01]
 [  7.51141497e-02]
 [  1.48848655e-01]
 [  1.46687456e-01]
 [  1.02690974e-01]
 [  7.92692819e-02]
 [  1.63024211e-01]
 [  1.55163246e-01]
 [  4.94283835e-02]
 [  1.55606427e-01]
 [  1.58285791e-01]
 [  1.64021391e-01]
 [  3.00780024e-02]
 [  1.54568563e-01]
 [  4.64459167e-02]
 [  8.76836958e-03]
 [  1.59833186e-01]
 [  5.42480041e-02]
 [  9.85040917e-02]
 [  7.46444787e-02]
 [  1.07763909e-01]
 [  2.40573762e-02]
 [  8.30958289e-02]
 [  2.63284935e-02]
 [  1.07775039e-01]
 [  4.16659309e-03]
 [  4.70025408e-02]
 [  2.42198144e-02]
 [  9.09458627e-02]
 [  1.56682645e-01]
 [  1.76315898e-02]
 [  7.18402187e-02]
 [  4.35908868e-02]
 [  1.59637489e-01]
 [  3.77469329e-02]
 [  1.84429477e-02]
 [  8.40738315e-03]
 [  1.46291445e-02]


In [101]:
print v1

[[  1.18393937e-01]
 [  4.35110900e-02]
 [  1.29056891e-01]
 [  1.60353070e-01]
 [  1.53917481e-02]
 [  4.01601069e-02]
 [  7.86243162e-02]
 [  8.91802072e-02]
 [  2.60132418e-03]
 [  1.02866304e-01]
 [  1.13983905e-01]
 [  7.51141497e-02]
 [  1.48848655e-01]
 [  1.46687456e-01]
 [  1.02690974e-01]
 [  7.92692819e-02]
 [  1.63024211e-01]
 [  1.55163246e-01]
 [  4.94283835e-02]
 [  1.55606427e-01]
 [  1.58285791e-01]
 [  1.64021391e-01]
 [  3.00780024e-02]
 [  1.54568563e-01]
 [  4.64459167e-02]
 [  8.76836958e-03]
 [  1.59833186e-01]
 [  5.42480041e-02]
 [  9.85040917e-02]
 [  7.46444787e-02]
 [  1.07763909e-01]
 [  2.40573762e-02]
 [  8.30958289e-02]
 [  2.63284935e-02]
 [  1.07775039e-01]
 [  4.16659309e-03]
 [  4.70025408e-02]
 [  2.42198144e-02]
 [  9.09458627e-02]
 [  1.56682645e-01]
 [  1.76315898e-02]
 [  7.18402187e-02]
 [  4.35908868e-02]
 [  1.59637489e-01]
 [  3.77469329e-02]
 [  1.84429477e-02]
 [  8.40738315e-03]
 [  1.46291445e-02]
 [  1.59027436e-01]
 [  3.84596779e-02]


In [106]:
d = np.diag(v1[:,0])
print d

[[ 0.11839394  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.04351109  0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.12905689 ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.14840788  0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.00121651  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.04947046]]


In [107]:
b = [0]*110
b = np.transpose(b)
print b.shape

(110,)


In [108]:
# now we computed v1, we want to compute v2, and then up to v_k
'''
    solve the linear equation
    dot product equals to zero means they are perpendicular
'''
print b.shape
v2 = np.linalg.solve(d,b)
print v2


(110,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
